<a href="https://colab.research.google.com/github/andyradz/aplios.solutions/blob/master/introduce.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AR-DiplomaWorkspace

#### MOUNT GOOGLE DRIVE VOLUME

In [27]:
## MOUNT GOOGLE DRIVE VOLUME

from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


#### LIST GOOGLE DRIVE VOLUME

In [28]:
## LIST GOOGLE DRIVE VOLUME
!ls -laFtA --full-time /content/gdrive 

### CREATE REQUIRED DATA FOLDERS

!mkdir input
!mkdir output

total 16
dr-x------  2 root root 4096 2022-11-25 07:32:08.000000000 +0000 .file-revisions-by-id/
drwx------ 22 root root 4096 2022-11-25 07:32:08.000000000 +0000 MyDrive/
dr-x------  2 root root 4096 2022-11-25 07:32:08.000000000 +0000 .shortcut-targets-by-id/
drwx------  5 root root 4096 2022-11-25 07:32:08.000000000 +0000 .Trash-0/


In [29]:
### DOWNLOAD AND INSTALL JAVA VIRTUAL MACHINE(JVM)

!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [30]:
### DOWNLOAD APACHE SPARK AND HADOOP ENVIRONMENT
!wget https://dlcdn.apache.org/spark/spark-3.3.1/spark-3.3.1-bin-hadoop3.tgz > /dev/null

### UNZIP SPARK ENVIRONMENT
!tar xf spark-3.3.1-bin-hadoop3.tgz > /dev/null

--2022-11-25 07:32:26--  https://dlcdn.apache.org/spark/spark-3.3.1/spark-3.3.1-bin-hadoop3.tgz
Resolving dlcdn.apache.org (dlcdn.apache.org)... 151.101.2.132, 2a04:4e42::644
Connecting to dlcdn.apache.org (dlcdn.apache.org)|151.101.2.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 299350810 (285M) [application/x-gzip]
Saving to: ‘spark-3.3.1-bin-hadoop3.tgz’

spark-3.3.1-bin-had 100%[===================>] 285.48M   175MB/s    in 1.6s    

2022-11-25 07:32:28 (175 MB/s) - ‘spark-3.3.1-bin-hadoop3.tgz’ saved [299350810/299350810]



In [31]:
### SET SYSTEM VARIABLES FOR PYSPARK

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = '/content/spark-3.3.1-bin-hadoop3'
os.environ['PYSPARK_SUBMIT_ARGS'] = '---jars /content/spark-3.3.1-bin-hadoop3/jars/spark-xml_2.12-0.11.0.jar pyspark-shell'

In [32]:
### INSTALL PYSPARK ENVIRONMENT
!pip install -q pyspark > /dev/null

### INSTALL LIBRARY FIND PYSPARK
!pip install -q findspark > /dev/null

### INITIAL PYSPARK ENVIRONMENT
import findspark
findspark.init()

### CHECK PYSPARK LOCATION
findspark.find()


'/content/spark-3.3.1-bin-hadoop3'

In [33]:
!sudo apt install --assume-yes p7zip-full

!type(7z)

Reading package lists... Done
Building dependency tree       
Reading state information... Done
p7zip-full is already the newest version (16.02+dfsg-6).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 5 not upgraded.
/bin/bash: -c: line 0: syntax error near unexpected token `7z'
/bin/bash: -c: line 0: `type(7z)'


In [34]:
### LIST PYTHON INSTALLED PACKAGES

!pip3 list -v


Package                       Version                Location                               Installer
----------------------------- ---------------------- -------------------------------------- ---------
absl-py                       1.3.0                  /usr/local/lib/python3.7/dist-packages pip
aeppl                         0.0.33                 /usr/local/lib/python3.7/dist-packages pip
aesara                        2.7.9                  /usr/local/lib/python3.7/dist-packages pip
aiohttp                       3.8.3                  /usr/local/lib/python3.7/dist-packages pip
aiosignal                     1.3.1                  /usr/local/lib/python3.7/dist-packages pip
alabaster                     0.7.12                 /usr/local/lib/python3.7/dist-packages pip
albumentations                1.2.1                  /usr/local/lib/python3.7/dist-packages pip
altair                        4.2.0                  /usr/local/lib/python3.7/dist-packages pip
appdirs                     

# Download stackexchange data 

In [35]:
#!FILE=/content/input/books.xml
#!if ![-f "$FILE"]; then
#!    echo Start downloading file $FILE
#!    wget https://github.com/databricks/spark-xml/raw/master/src/test/resources/books.xml  -P /content/input/ > /dev/null
#!fi


!wget https://archive.org/download/stackexchange/mathematica.stackexchange.com.7z -P /content/input/ > /dev/null

!7za e -y /content/input/mathematica.stackexchange.com.7z -o/content/input/ > /dev/null

--2022-11-25 07:33:20--  https://archive.org/download/stackexchange/mathematica.stackexchange.com.7z
Resolving archive.org (archive.org)... 207.241.224.2
Connecting to archive.org (archive.org)|207.241.224.2|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://ia800107.us.archive.org/27/items/stackexchange/mathematica.stackexchange.com.7z [following]
--2022-11-25 07:33:21--  https://ia800107.us.archive.org/27/items/stackexchange/mathematica.stackexchange.com.7z
Resolving ia800107.us.archive.org (ia800107.us.archive.org)... 207.241.232.17
Connecting to ia800107.us.archive.org (ia800107.us.archive.org)|207.241.232.17|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 265738334 (253M) [application/x-7z-compressed]
Saving to: ‘/content/input/mathematica.stackexchange.com.7z’

mathematica.stackex 100%[===================>] 253.43M  18.5MB/s    in 10s     

2022-11-25 07:33:31 (24.9 MB/s) - ‘/content/input/mathematica.stackexchange.com

In [10]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, LongType, StringType, BooleanType, DateType 
from pyspark.sql.functions import to_timestamp, date_format, col

spark = SparkSession.builder.master("local[4]").appName('SparkByExamples.com').config('spark.jars.packages', 'com.databricks:spark-xml_2.12:0.6.0').getOrCreate()

#### LOAD AND SAVE POSTS.XML DATASET TO PARQUET FORMAT

In [52]:
schema = StructType([ \
    StructField("_AnswerCount", LongType(), True), \
    StructField("_Body", StringType(), True), \
    StructField("_CommentCount", LongType(), True), \
    StructField("_CreationDate", StringType(), True), \
    StructField("_Id", LongType(), True), \
    StructField("_LastActivityDate", StringType(), True), \
    StructField("_OwnerUserId", LongType(), True), \
    StructField("_ParentId", LongType(), True), \
    StructField("_PostTypeId", LongType(), True), \
    StructField("_Score", LongType(), True), \
    StructField("_Tags", StringType(), True), \
    StructField("_Title", StringType(), True), \
    StructField("_ViewCount", LongType(), True), \
    StructField("Year", LongType(), True), \
    StructField("Month", LongType(), True), \
    StructField("Data", LongType(), True) \
]) 

postsDF = spark.read \
          .format('com.databricks.spark.xml') \
          .options(rootTag='posts') \
          .options(rowTag='row') \
          .schema(schema) \
          .load('/content/input/Posts.xml')     

postsDF = postsDF \
                .withColumnRenamed("_AnswerCount", "AnswerCount") \
                .withColumnRenamed("_Body", "Body") \
                .withColumnRenamed("_CommentCount", "CommentCount") \
                .withColumnRenamed("_CreationDate", "CreationDate") \
                .withColumnRenamed("_Id", "Id") \
                .withColumnRenamed("_LastActivityDate", "LastActivityDate") \
                .withColumnRenamed("_OwnerUserId", "OwnerUserId") \
                .withColumnRenamed("_ParentId", "ParentId") \
                .withColumnRenamed("_PostTypeId", "PostTypeId") \
                .withColumnRenamed("_Score", "Score") \
                .withColumnRenamed("_Tags", "Tags") \
                .withColumnRenamed("_Title", "Title") \
                .withColumnRenamed("_ViewCount", "ViewCount")

postsDF = postsDF \
  .select(["Id", "ParentId", "PostTypeId", "OwnerUserId", \
          to_timestamp("CreationDate").alias("CreationDate"), "Score", \
          "ViewCount", "AnswerCount", \
          "CommentCount", to_timestamp("LastActivityDate").alias("LastActivityDate"), \
          "Tags", "Title", "Body"]) \
  .withColumn("Year", date_format(col("CreationDate"), "y").cast("integer")) \
  .withColumn("Month", date_format(col("CreationDate"), "M").cast("integer")) \
  .withColumn("Day", date_format(col("CreationDate"), "d").cast("integer")) \

postsDF.printSchema()   
postsDF.show(n=10, truncate=False)

postsDF = postsDF \
      .write \
      .partitionBy("Year", "Month", "Day") \
      .mode("overwrite") \
      .parquet("/content/output/posts.parquet")      

root
 |-- Id: long (nullable = true)
 |-- ParentId: long (nullable = true)
 |-- PostTypeId: long (nullable = true)
 |-- OwnerUserId: long (nullable = true)
 |-- CreationDate: timestamp (nullable = true)
 |-- Score: long (nullable = true)
 |-- ViewCount: long (nullable = true)
 |-- AnswerCount: long (nullable = true)
 |-- CommentCount: long (nullable = true)
 |-- LastActivityDate: timestamp (nullable = true)
 |-- Tags: string (nullable = true)
 |-- Title: string (nullable = true)
 |-- Body: string (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Month: integer (nullable = true)
 |-- Day: integer (nullable = true)

+---+--------+----------+-----------+-----------------------+-----+---------+-----------+------------+-----------------------+---------------------------------------------------------------------------------+------------------------------------------------------------------------------------------------------------------+---------------------------------------------

#### LOAD AND SAVE BADGES.XML DATASET TO PARQUET FORMAT

In [ ]:
schema = StructType([ \
    StructField("_Class", LongType(), True), \
    StructField("_Date", StringType(), True), \
    StructField("_CommentCount", LongType(), True), \
    StructField("_Id", LongType(), True), \
    StructField("_Name", StringType(), True), \
    StructField("_TagBased", BooleanType(), True), \
    StructField("_UserId", LongType(), True)    
])

badgesDF = spark.read \
          .format('com.databricks.spark.xml') \
          .options(rootTag='badges') \
          .options(rowTag='row') \
          .schema(schema) \
          .load('/content/input/Badges.xml')

badgesDF = badgesDF \
                .withColumnRenamed("_Class", "Class") \
                .withColumnRenamed("_Date", "Date") \
                .withColumnRenamed("_CommentCount", "CommentCount") \
                .withColumnRenamed("_Id", "Id") \
                .withColumnRenamed("_Name", "Name") \
                .withColumnRenamed("_TagBased", "TagBased") \
                .withColumnRenamed("_UserId", "UserId")

badgesDF = badgesDF \
              .select(["Id", "UserId", "Class", "Date", "CommentCount", \
                      "TagBased","Name"])

badgesDF.printSchema()
badgesDF.show(n=10, truncate=False)

badgesDF \
      .write \
      .partitionBy("Class") \
      .mode("overwrite") \
      .parquet("/content/output/badges.parquet")

#### LOAD AND SAVE COMMENTS.XML DATASET TO PARQUET FORMAT

In [ ]:
schema = StructType([ \
    StructField("_ContentLicense", StringType(), True), \
    StructField("_CreationDate", StringType(), True), \
    StructField("_Id", LongType(), True), \
    StructField("_PostId", LongType(), True), \
    StructField("_Score", LongType(), True), \
    StructField("_Text", StringType(), True), \
    StructField("_UserDisplayName", StringType(), True), \
    StructField("_UserId", LongType(), True)        
])

commentsDF = spark.read \
          .format('com.databricks.spark.xml') \
          .options(rootTag='comments') \
          .options(rowTag='row') \
          .schema(schema) \
          .load('/content/input/Comments.xml')

commentsDF = commentsDF \
                .withColumnRenamed("_ContentLicense", "ContentLicense") \
                .withColumnRenamed("_CreationDate", "CreationDate") \
                .withColumnRenamed("_Id", "Id") \
                .withColumnRenamed("_PostId", "PostId") \
                .withColumnRenamed("_Score", "Score") \
                .withColumnRenamed("_Text", "Text") \
                .withColumnRenamed("_UserDisplayName", "UserDisplayName") \
                .withColumnRenamed("_UserId", "UserId") \

commentsDF = commentsDF \
  .select(["Id", "PostId", "UserId", "CreationDate", "ContentLicense", \
           "UserDisplayName", "Score", "Text"])

commentsDF.printSchema()
commentsDF.show(n=10, truncate=False)

commentsDF \
    .write \
    .partitionBy("ContentLicense") \
    .mode("overwrite") \
    .parquet("/content/output/comments.parquet")

#### LOAD AND SAVE POSTHISTORY.XML DATASET TO PARQUET FORMAT

In [ ]:
schema = StructType([ \
    StructField("_Comment", StringType(), True), \
    StructField("_ContentLicense", StringType(), True), \
    StructField("_CreationDate", StringType(), True), \
    StructField("_Id", LongType(), True), \
    StructField("_PostHistoryTypeId", LongType(), True), \
    StructField("_PostId", LongType(), True), \
    StructField("_RevisionGUID", StringType(), True), \
    StructField("_Text", StringType(), True), \
    StructField("_UserDisplayName", StringType(), True), \
    StructField("_UserId", LongType(), True)    
])

postHistoryDF = spark.read \
          .format('com.databricks.spark.xml') \
          .options(rootTag='posthistory') \
          .options(rowTag='row') \
          .schema(schema) \
          .load('/content/input/PostHistory.xml')
               
postHistoryDF = postHistoryDF \
              .withColumnRenamed("_Comment", "Comment") \
              .withColumnRenamed("_ContentLicense", "ContentLicense") \
              .withColumnRenamed("_CreationDate", "CreationDate") \
              .withColumnRenamed("_Id", "Id") \
              .withColumnRenamed("_PostHistoryTypeId", "PostHistoryTypeId") \
              .withColumnRenamed("_PostId", "PostId") \
              .withColumnRenamed("_RevisionGUID", "RevisionGUID") \
              .withColumnRenamed("_Text", "Text") \
              .withColumnRenamed("_UserDisplayName", "UserDisplayName") \
              .withColumnRenamed("_UserId", "UserId") \

postHistoryDF = postHistoryDF \
              .select(["Id", "PostId", "PostHistoryTypeId", "UserId", \
                       "CreationDate", "RevisionGUID", "ContentLicense", "UserDisplayName", "Comment", "Text"])

postHistoryDF.printSchema()
postHistoryDF.show(n=10, truncate=False)  

postHistoryDF \
  .write \
  .partitionBy("ContentLicense") \
  .mode("overwrite") \
  .parquet("/content/output/posthistory.parquet")  

#### LOAD AND SAVE TAGS.XML DATASET TO PARQUET FORMAT

In [ ]:
schema = StructType([ \
    StructField("_Count", LongType(), True), \
    StructField("_ExcerptPostId", LongType(), True), \
    StructField("_Id", LongType(), True), \
    StructField("_TagName", StringType(), True), \
    StructField("_WikiPostId", LongType(), True)    
])

tagsDF = spark.read \
          .format('com.databricks.spark.xml') \
          .options(rootTag='tags') \
          .options(rowTag='row') \
          .schema(schema) \
          .load('/content/input/Tags.xml')

tagsDF = tagsDF \
        .withColumnRenamed("_Count", "Count") \
        .withColumnRenamed("_ExcerptPostId", "ExcerptPostId") \
        .withColumnRenamed("_Id", "Id") \
        .withColumnRenamed("_TagName", "TagName") \
        .withColumnRenamed("_WikiPostId", "WikiPostId")

tagsDF = tagsDF \
        .select(["Id", "ExcerptPostId", "WikiPostId", "Count", "TagName"])

tagsDF.printSchema()
tagsDF.show(n=10, truncate=False)

tagsDF \
  .write \
  .partitionBy("Id") \
  .mode("overwrite") \
  .parquet("/content/output/tags.parquet") 

#### LOAD AND SAVE USERS.XML DATASET TO PARQUET FORMAT

In [ ]:
from pyspark.sql.functions import *

schema = StructType([ \
    StructField("_AboutMe", StringType(), True), \
    StructField("_AccountId", LongType(), True), \
    StructField("_CreationDate", StringType(), True), \
    StructField("_DisplayName", StringType(), True), \
    StructField("_DownVotes", LongType(), True), \
    StructField("_Id", LongType(), True), \
    StructField("_LastAccessDate", StringType(), True), \
    StructField("_Location", StringType(), True), \
    StructField("_ProfileImageUrl", StringType(), True), \
    StructField("_Reputation", LongType(), True), \
    StructField("_UpVotes", LongType(), True), \
    StructField("_Views", LongType(), True), \
    StructField("_WebsiteUrl", StringType(), True)
])

usersDF = spark.read \
          .format('com.databricks.spark.xml') \
          .options(rootTag='Users') \
          .options(rowTag='row') \
          .schema(schema) \
          .load('/content/input/Users.xml')

usersDF = usersDF \
  .withColumnRenamed("_AboutMe", "AboutMe") \
  .withColumnRenamed("_AccountId", "AccountId") \
  .withColumnRenamed("_CreationDate", "CreationDate") \
  .withColumnRenamed("_DisplayName", "DisplayName") \
  .withColumnRenamed("_DownVotes", "DownVotes") \
  .withColumnRenamed("_Id", "Id") \
  .withColumnRenamed("_LastAccessDate", "LastAccessDate") \
  .withColumnRenamed("_Location", "Location") \
  .withColumnRenamed("_ProfileImageUrl", "ProfileImageUrl") \
  .withColumnRenamed("_Reputation", "Reputation") \
  .withColumnRenamed("_UpVotes", "UpVotes") \
  .withColumnRenamed("_Views", "Views") \
  .withColumnRenamed("_WebsiteUrl", "WebsiteUrl") \

usersDF = usersDF \
  .select(["Id", "AccountId", to_timestamp("CreationDate").alias("CreationDate"), \
           to_timestamp("LastAccessDate").alias("LastAccessDate"), "DownVotes", \
           "UpVotes", "Reputation", "Views", "WebsiteUrl", "ProfileImageUrl", \
           "Location", "DisplayName", "AboutMe"           
           ]) \
  .withColumn("Year", date_format(col("CreationDate"), "y")) \
  .withColumn("Month", date_format(col("CreationDate"), "M")) \
  .withColumn("Day", date_format(col("CreationDate"), "d"))

usersDF.printSchema()
usersDF.show(n=10, truncate=False) 

usersDF \
  .write \
  .partitionBy("Year", "Month", "Day") \
  .mode("overwrite") \
  .parquet("/content/output/users.parquet")

#### LOAD AND SAVE VOTES.XML DATASET TO PARQUET FORMAT

In [18]:
schema = StructType([ \
    StructField("_BountyAmount", LongType(), True), \
    StructField("_CreationDate", StringType(), True), \
    StructField("_Id", LongType(), True), \
    StructField("_PostId", LongType(), True), \
    StructField("_UserId", LongType(), True), \
    StructField("_VoteTypeId", LongType(), True)  
])

votesDF = spark.read \
          .format('com.databricks.spark.xml') \
          .options(rootTag='votes') \
          .options(rowTag='row') \
          .schema(schema) \
          .load('/content/input/Votes.xml')

votesDF = votesDF \
  .withColumnRenamed("_BountyAmount", "BountyAmount") \
  .withColumnRenamed("_CreationDate", "CreationDate") \
  .withColumnRenamed("_Id", "Id") \
  .withColumnRenamed("_PostId", "PostId") \
  .withColumnRenamed("_UserId", "UserId") \
  .withColumnRenamed("_VoteTypeId", "VoteTypeId") \

votesDF = votesDF \
  .select(["Id", "PostId", "UserId", "VoteTypeId", \
          to_timestamp("CreationDate").alias("CreationDate"), "BountyAmount"]) \
  .withColumn("Year", date_format(col("CreationDate"), "y")) \
  .withColumn("Month", date_format(col("CreationDate"), "M")) \
  .withColumn("Day", date_format(col("CreationDate"), "d"))  

votesDF.printSchema()
votesDF.show(n=10, truncate=False) 

votesDF \
  .write \
  .mode("overwrite") \
  .parquet("/content/output/votes.parquet")

root
 |-- Id: long (nullable = true)
 |-- PostId: long (nullable = true)
 |-- UserId: long (nullable = true)
 |-- VoteTypeId: long (nullable = true)
 |-- CreationDate: timestamp (nullable = true)
 |-- BountyAmount: long (nullable = true)
 |-- Year: string (nullable = true)
 |-- Month: string (nullable = true)
 |-- Day: string (nullable = true)

+---+------+------+----------+-------------------+------------+----+-----+---+
|Id |PostId|UserId|VoteTypeId|CreationDate       |BountyAmount|Year|Month|Day|
+---+------+------+----------+-------------------+------------+----+-----+---+
|1  |1     |null  |2         |2012-01-17 00:00:00|null        |2012|1    |17 |
|2  |1     |null  |2         |2012-01-17 00:00:00|null        |2012|1    |17 |
|3  |1     |null  |2         |2012-01-17 00:00:00|null        |2012|1    |17 |
|4  |1     |null  |2         |2012-01-17 00:00:00|null        |2012|1    |17 |
|5  |1     |null  |2         |2012-01-17 00:00:00|null        |2012|1    |17 |
|6  |2     |null  |2 

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/usr/local/lib/python3.7/dist-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.7/socket.py", line 589, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: ignored

In [63]:
### ANALITYKA POSTS

postsDF1 = spark.read.parquet("/content/output/posts.parquet")

postsDF1 = postsDF1 \
  .filter(col("Year") == 2011) \
  .groupBy("Year", "Month") \
  .count() \
  .orderBy(col("Year").desc(), col("Month").desc()) \
  .withColumnRenamed("count", "Count") \
  .show(n=100, truncate=False)

# postsDF1 \
#   .withColumn('word', explode(f.split(f.col('Description'), ' ')))\
#   .groupBy('word')\
#   .count()\
#   .sort('count', ascending=False)\
#   .show()

# import pandas as pd
# import matplotlib.pyplot as plt

# plotdata = pd.DataFrame({
#     "2010":[18,13,10,19, 11], #MIESIĄC/ILOŚĆ
#     "2017":[7,7,7,3, 33],
#     "2018":[57,67,77,83, 32],
#     "2019":[68,73,80,79, 32],    
#     "2020":[73,78,80,85, 34]},
#     index=["Django", "Gafur", "Tommy", "Ronnie", "Python"]) #LATA
# plotdata.plot(kind='bar', stacked=True, figsize=(15, 8))

# plt.title("Post ratings")

# plt.xlabel("Years")

# plt.ylabel("Number of posts")    

#plt.bar(df['Year'], df['Sample Size'])
#plt.plot(plotdata['2017'], plotdata[7,7,7,3], '-o', color='orange')
#https://www.dataquest.io/blog/how-to-plot-a-bar-graph-matplotlib/

# from plotly.subplots import make_subplots

# speed = [0.1, 17.5, 40, 48, 52, 69, 88]
# lifespan = [2, 8, 70, 1.5, 25, 12, 28]
# index = ['snail', 'pig', 'elephant', 'rabbit', 'giraffe', 'coyote', 'horse']
# df = pd.DataFrame({'speed': speed,'lifespan': lifespan}, index=index)
# df.plot.bar()


+----+-----+-----+
|Year|Month|Count|
+----+-----+-----+
|2011|12   |12   |
|2011|11   |13   |
|2011|10   |9    |
|2011|9    |3    |
|2011|8    |8    |
|2011|7    |16   |
|2011|6    |5    |
|2011|5    |5    |
|2011|4    |6    |
|2011|3    |10   |
|2011|2    |1    |
|2011|1    |6    |
+----+-----+-----+



In [ ]:
#GRUPOWANIE 

import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col,sum,avg,max

spark = SparkSession.builder.appName('SparkByExamples.com').getOrCreate()

simpleData = [("James","Sales","NY",90000,34,10000),
    ("Michael","Sales","NY",86000,56,20000),
    ("Robert","Sales","CA",81000,30,23000),
    ("Maria","Finance","CA",90000,24,23000),
    ("Raman","Finance","CA",99000,40,24000),
    ("Scott","Finance","NY",83000,36,19000),
    ("Jen","Finance","NY",79000,53,15000),
    ("Jeff","Marketing","CA",80000,25,18000),
    ("Kumar","Marketing","NY",91000,50,21000)
  ]

schema = ["employee_name","department","state","salary","age","bonus"]
df = spark.createDataFrame(data=simpleData, schema = schema)
df.printSchema()
df.show(truncate=False)

df.groupBy("department").sum("salary").show(truncate=False)

df.groupBy("department").count().show(truncate=False)


df.groupBy("department","state") \
    .sum("salary","bonus") \
   .show(truncate=False)

df.groupBy("department") \
    .agg(sum("salary").alias("sum_salary"), \
         avg("salary").alias("avg_salary"), \
         sum("bonus").alias("sum_bonus"), \
         max("bonus").alias("max_bonus") \
     ) \
    .show(truncate=False)
    
df.groupBy("department") \
    .agg(sum("salary").alias("sum_salary"), \
      avg("salary").alias("avg_salary"), \
      sum("bonus").alias("sum_bonus"), \
      max("bonus").alias("max_bonus")) \
    .where(col("sum_bonus") >= 50000) \
    .show(truncate=False)